In [1]:
import pandas as pd
import numpy as np
from sklearn import cross_validation, metrics   #Additional scklearn functions
from sklearn.grid_search import GridSearchCV   #Perforing grid search
from sklearn.preprocessing import LabelEncoder
from pandas.api.types import CategoricalDtype

/home/lucio/Documentos/Datos/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/lucio/Documentos/Datos/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
dtypes = pd.Series({'sepostulo': np.dtype('uint8'),'edad': np.dtype('uint16')})

dtypes_col = dtypes.index
dtypes_type = [i.name for i in dtypes.values]

column_types = dict(zip(dtypes_col, dtypes_type))

trainingSet = pd.read_csv('/home/lucio/Documentos/Datos/NaventDatosTP/Data/TRAINING_SET_SIN_ENCODING.csv',dtype=column_types)
trainingSet.drop(['Unnamed: 0','denominacion_empresa'],axis=1,inplace=True)
trainingSet[["sexo",'nombre','estado','nombre_zona',
             'tipo_de_trabajo','nivel_laboral',
             'nombre_area']] = trainingSet[["sexo",'nombre','estado','nombre_zona',
                                            'tipo_de_trabajo','nivel_laboral',
                                            'nombre_area']].astype('category')

# Me quedo con 1.5M de registros random

In [3]:
trainingSet_samples = trainingSet.sample(n=2500000,random_state=50)

In [4]:
trainingSet_samples['sepostulo'].value_counts()

1    1571165
0     928835
Name: sepostulo, dtype: int64

# Hago Dummificacion

In [5]:
trainingSet_samples = trainingSet_samples.join(pd.get_dummies(trainingSet_samples.sexo))

trainingSet_samples.drop('sexo',axis=1, inplace=True)

trainingSet_samples = trainingSet_samples.join(pd.get_dummies(trainingSet_samples.nombre))

trainingSet_samples.drop('nombre',axis=1, inplace=True)

trainingSet_samples = trainingSet_samples.join(pd.get_dummies(trainingSet_samples.estado))

trainingSet_samples.drop('estado',axis=1, inplace=True)

trainingSet_samples = trainingSet_samples.join(pd.get_dummies(trainingSet_samples.nombre_zona))

trainingSet_samples.drop('nombre_zona',axis=1, inplace=True)

trainingSet_samples = trainingSet_samples.join(pd.get_dummies(trainingSet_samples.tipo_de_trabajo))

trainingSet_samples.drop('tipo_de_trabajo',axis=1, inplace=True)

trainingSet_samples = trainingSet_samples.join(pd.get_dummies(trainingSet_samples.nivel_laboral,
                                                             prefix="Nivel_laboral"))

trainingSet_samples.drop('nivel_laboral',axis=1, inplace=True)

#trainingSet_samples = trainingSet_samples.join(pd.get_dummies(trainingSet_samples.nombre_area))
# LO SAQUE PORQUE MI MAQUINA NO SE BANCA POR AHI LA DE UDS SI AUNQUE AGREGA MUCHAS COLUMAS(188)

trainingSet_samples.drop('nombre_area',axis=1, inplace=True)

In [8]:
from sklearn import metrics
import lightgbm as lgb

from sklearn.model_selection import train_test_split
columnas = trainingSet_samples.columns.drop("sepostulo")
xtrain, xtest, ytrain, ytest = train_test_split(trainingSet_samples[columnas], 
                                        trainingSet_samples['sepostulo'],test_size=0.30, random_state=12)


d_train = lgb.Dataset(xtrain, label=ytrain)

params = {}
params['learning_rate'] = 0.003
params['boosting_type'] = 'rf'
params['objective'] = 'binary'
params['metric'] = 'binary_logloss'
params['sub_feature'] = 0.5
params['num_leaves'] = 10
params['min_data'] = 50
params['max_depth'] = 10
params['is_unbalanced'] = True
# Train and evaluate.
clf = lgb.train(params, d_train, 100)
y_pred = clf.predict(xtest)  #predicts

print('confusion matrix')
print(metrics.confusion_matrix(ytest, y_pred.round()))
print('classification report')
print(metrics.classification_report(ytest, y_pred.round()))
print("-----------------------------------------------------------------------------------------")
print("Accuracy is :")
print(metrics.accuracy_score(ytest, y_pred.round(), normalize=False))
print('Area under the curve : %f' % (metrics.roc_auc_score(ytest, y_pred.round())))

LightGBMError: Check failed: config->bagging_freq > 0 && config->bagging_fraction < 1.0f && config->bagging_fraction > 0.0f at /home/travis/build/Microsoft/LightGBM/python-package/compile/src/boosting/rf.hpp, line 29 .
